# Windows-Clippy-MCP Integration with AG2

This notebook demonstrates the integration between AG2 and Windows-Clippy-MCP, including:
- VSCode extension support
- Azure Key Vault integration for secure API key management
- Entra ID authentication
- Memory engine with agent context
- Natural language interface for AG2 tooling

**Authors:** AG2 Team  
**Integration Target:** [Windows-Clippy-MCP](https://github.com/dayour/Windows-Clippy-MCP)

## Prerequisites

Before running this notebook, ensure you have:

1. **Installed AG2 with Windows-Clippy-MCP support:**
   ```bash
   pip install -e ".[windows-clippy-mcp]"
   ```

2. **Set up Azure credentials** (optional, for Key Vault integration):
   - Azure Tenant ID
   - Application Client ID
   - Client Secret or Certificate
   - Key Vault URL

3. **Configured environment variables** or use the provided Azure Key Vault integration

In [ ]:
import os
import asyncio
import json
from pathlib import Path
from typing import Dict, List, Optional

# Import AG2 core components
from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat

# Import Windows-Clippy-MCP integration
from autogen.mcp.clippy_mcp import (
    ClippyMCPConfig,
    WindowsClippyMCPClient,
    create_clippy_toolkit,
    AzureKeyVaultConfig,
    EntraIDConfig
)

print("✓ All imports successful")

## Configuration Setup

Set up the Windows-Clippy-MCP configuration with Azure integration.

In [ ]:
# Basic configuration - update with your settings
AZURE_TENANT_ID = os.getenv("AZURE_TENANT_ID", "your-tenant-id")
AZURE_CLIENT_ID = os.getenv("AZURE_CLIENT_ID", "your-client-id")
AZURE_CLIENT_SECRET = os.getenv("AZURE_CLIENT_SECRET")
AZURE_KEY_VAULT_URL = os.getenv("AZURE_KEY_VAULT_URL", "https://your-vault.vault.azure.net/")

# Configure Azure Key Vault (optional)
azure_keyvault_config = AzureKeyVaultConfig(
    vault_url=AZURE_KEY_VAULT_URL,
    tenant_id=AZURE_TENANT_ID,
    client_id=AZURE_CLIENT_ID,
    use_managed_identity=False  # Set to True if using managed identity
)

# Configure Entra ID authentication
entra_id_config = EntraIDConfig(
    tenant_id=AZURE_TENANT_ID,
    client_id=AZURE_CLIENT_ID,
    client_secret=AZURE_CLIENT_SECRET,
    scopes=["https://graph.microsoft.com/.default"]
)

# Main Windows-Clippy-MCP configuration
clippy_config = ClippyMCPConfig(
    server_name="ag2-windows-clippy",
    clippy_executable_path="windows-clippy-mcp",  # Update with actual path
    
    # Azure integrations
    azure_key_vault=azure_keyvault_config if AZURE_CLIENT_SECRET else None,
    entra_id=entra_id_config if AZURE_CLIENT_SECRET else None,
    
    # Shared LLM keys (can be overridden by Key Vault)
    shared_llm_keys={
        "openai": os.getenv("OPENAI_API_KEY", ""),
        "anthropic": os.getenv("ANTHROPIC_API_KEY", ""),
        "azure-openai": os.getenv("AZURE_OPENAI_API_KEY", "")
    },
    
    # Key Vault mapping for LLM services
    llm_key_vault_mapping={
        "openai": "openai-api-key",
        "anthropic": "anthropic-api-key",
        "azure-openai": "azure-openai-api-key"
    },
    
    # VSCode extension settings
    vscode_extension_enabled=True,
    vscode_extension_port=8765,
    
    # Memory engine
    memory_engine_enabled=True,
    memory_storage_type="azure"
)

print("✓ Configuration created")
print(f"  Server: {clippy_config.server_name}")
print(f"  VSCode Extension: {'Enabled' if clippy_config.vscode_extension_enabled else 'Disabled'}")
print(f"  Memory Engine: {'Enabled' if clippy_config.memory_engine_enabled else 'Disabled'}")
print(f"  Azure Integration: {'Enabled' if clippy_config.azure_key_vault else 'Disabled'}")

## Initialize Windows-Clippy-MCP Client

Create and initialize the Windows-Clippy-MCP client with Azure authentication.

In [ ]:
# Create the Windows-Clippy-MCP client
clippy_client = WindowsClippyMCPClient(clippy_config)

# Initialize Azure authentication (if configured)
await clippy_client.initialize_azure_auth()

print("✓ Windows-Clippy-MCP client initialized")

# Test LLM key retrieval
test_services = ["openai", "anthropic", "azure-openai"]
for service in test_services:
    key = await clippy_client.get_llm_api_key(service)
    status = "✓ Available" if key else "⚠ Not configured"
    print(f"  {service}: {status}")

## Create Windows-Clippy-MCP Toolkit

Create a toolkit that integrates AG2 with Windows-Clippy-MCP functionality.

In [ ]:
# Create the toolkit with Windows-Clippy-MCP integration
try:
    clippy_toolkit = await create_clippy_toolkit(
        config=clippy_config,
        use_mcp_tools=True,
        use_mcp_resources=True,
        resource_download_folder=Path("./downloads")
    )
    
    print("✓ Windows-Clippy-MCP toolkit created")
    print(f"  Available tools: {len(clippy_toolkit.tools)}")
    
    # List available tools
    for tool_name, tool in clippy_toolkit.tools.items():
        print(f"  - {tool_name}: {tool.description[:60]}...")
        
except Exception as e:
    print(f"⚠ Could not create toolkit (MCP server may not be running): {e}")
    print("This is expected if the Windows-Clippy-MCP server is not yet installed.")
    clippy_toolkit = None

## Create AG2 Agents with Windows-Clippy-MCP Integration

Set up AG2 agents that can leverage the Windows-Clippy-MCP functionality.

In [ ]:
# Configure LLM (using retrieved API key if available)
openai_key = await clippy_client.get_llm_api_key("openai")
llm_config = LLMConfig({
    "model": "gpt-4",
    "api_key": openai_key or os.getenv("OPENAI_API_KEY"),
    "temperature": 0.7
})

# Create a Clippy Assistant agent
clippy_assistant = ConversableAgent(
    name="ClippyAssistant",
    system_message="""You are Clippy, the helpful AI assistant integrated with AG2 and Windows.
    You can help with:
    - Creating and managing AG2 agents
    - Running multi-agent conversations
    - Executing code and managing development tasks
    - Storing and retrieving information from memory
    - Integrating with VSCode and other development tools
    
    Always be helpful, friendly, and provide clear explanations of what you're doing.""",
    llm_config=llm_config,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3
)

# Create a Developer agent
developer_agent = ConversableAgent(
    name="Developer",
    system_message="""You are an experienced software developer who works with AG2.
    You focus on:
    - Writing and reviewing code
    - Architecting multi-agent systems
    - Debugging and optimization
    - Best practices and patterns
    
    Collaborate with Clippy to build effective solutions.""",
    llm_config=llm_config,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3
)

# Register the Windows-Clippy-MCP toolkit with agents (if available)
if clippy_toolkit:
    clippy_toolkit.register_for_llm(clippy_assistant)
    clippy_toolkit.register_for_execution(clippy_assistant)
    
    clippy_toolkit.register_for_llm(developer_agent)
    clippy_toolkit.register_for_execution(developer_agent)
    
    print("✓ Toolkit registered with agents")
else:
    print("⚠ Toolkit not available - agents created without Windows-Clippy-MCP tools")

print("✓ Agents created:")
print(f"  - {clippy_assistant.name}: Clippy Assistant with Windows integration")
print(f"  - {developer_agent.name}: Development-focused agent")

## Demonstrate Windows-Clippy-MCP Features

Show the key features of the Windows-Clippy-MCP integration.

In [ ]:
# Demonstrate memory functionality
print("=== Memory Engine Demo ===")

# Simulate saving to memory (would use actual MCP tools if server is running)
memory_demo = {
    "project_context": "Working on AG2 Windows-Clippy-MCP integration",
    "key_features": [
        "VSCode extension support",
        "Azure Key Vault integration", 
        "Entra ID authentication",
        "Memory engine with agent context"
    ],
    "agent_preferences": {
        "ClippyAssistant": "Helpful and friendly explanations",
        "Developer": "Focus on code quality and best practices"
    }
}

print("Memory context stored:")
print(json.dumps(memory_demo, indent=2))

print("\n=== VSCode Integration Demo ===")
vscode_config = clippy_client.create_vscode_extension_config()
print("VSCode extension configuration:")
print(f"  Name: {vscode_config['name']}")
print(f"  Publisher: {vscode_config['publisher']}")
print(f"  Commands: {len(vscode_config['contributes']['commands'])} available")

print("\n=== Agent Collaboration Demo ===")
# Simple conversation between agents
initial_message = """Let's work on creating a multi-agent system that can:
1. Monitor Windows system performance
2. Suggest optimizations based on usage patterns
3. Integrate with development tools like VSCode
4. Store insights in our memory engine for future reference

What would be a good architecture for this?"""

# Start conversation
response = await clippy_assistant.a_generate_reply(
    messages=[{"content": initial_message, "role": "user"}]
)

print(f"Clippy Assistant: {response}")

# Developer response
dev_response = await developer_agent.a_generate_reply(
    messages=[
        {"content": initial_message, "role": "user"},
        {"content": str(response), "role": "assistant"}
    ]
)

print(f"\nDeveloper: {dev_response}")

## VSCode Extension Setup

Generate the VSCode extension files for Windows-Clippy-MCP integration.

In [ ]:
from autogen.mcp.clippy_mcp import create_vscode_extension_files

# Create VSCode extension files
extension_dir = Path("./vscode-ag2-clippy-extension")
extension_dir.mkdir(exist_ok=True)

try:
    create_vscode_extension_files(clippy_config, extension_dir)
    print(f"✓ VSCode extension files created in {extension_dir}")
    
    # List created files
    for file_path in extension_dir.rglob("*"):
        if file_path.is_file():
            relative_path = file_path.relative_to(extension_dir)
            print(f"  - {relative_path}")
    
    print("\nTo install the extension:")
    print(f"1. cd {extension_dir}")
    print("2. npm install")
    print("3. npm run compile")
    print("4. Install in VSCode using 'Developer: Install Extension from Location'")
    
except Exception as e:
    print(f"Error creating extension files: {e}")

## Deployment Instructions

Instructions for deploying the complete Windows-Clippy-MCP integration.

In [ ]:
deployment_instructions = """
🚀 Windows-Clippy-MCP Deployment Guide
=====================================

1. AUTOMATED DEPLOYMENT:
   Run the deployment script:
   ```bash
   python scripts/deploy_windows_clippy_mcp.py
   ```
   
   Options:
   --no-uv          Use pip instead of UV
   --no-vscode      Skip VSCode extension setup
   --no-azure       Skip Azure integration setup
   --port 8765      Set MCP server port

2. MANUAL SETUP:
   a) Install AG2 with Windows-Clippy-MCP support:
      pip install -e ".[windows-clippy-mcp]"
   
   b) Set up environment variables:
      - AZURE_TENANT_ID
      - AZURE_CLIENT_ID  
      - AZURE_CLIENT_SECRET
      - AZURE_KEY_VAULT_URL
      - OPENAI_API_KEY (or store in Key Vault)
   
   c) Configure Azure Key Vault (optional):
      - Create secrets for LLM API keys
      - Grant application access to Key Vault
   
   d) Install VSCode extension:
      - Build extension files using this notebook
      - Run npm install && npm run compile
      - Install in VSCode

3. START THE MCP SERVER:
   ```bash
   python autogen/mcp/ag2_mcp_server.py sse --port 8765
   ```
   
   Or for stdio mode:
   ```bash
   python autogen/mcp/ag2_mcp_server.py stdio
   ```

4. CONFIGURE MCP CLIENTS:
   - VSCode: Use the installed extension
   - Claude Desktop: Add server to configuration
   - Other MCP clients: Connect to http://localhost:8765/sse

5. VERIFY INSTALLATION:
   - Check that agents can be created
   - Test memory storage and retrieval
   - Verify Azure Key Vault integration (if configured)
   - Test VSCode extension commands
"""

print(deployment_instructions)

## Summary

This notebook demonstrated the Windows-Clippy-MCP integration with AG2, including:

### ✅ Implemented Features:
- **Windows-Clippy-MCP Client**: Full integration with configuration management
- **Azure Key Vault Integration**: Secure API key storage and retrieval
- **Entra ID Authentication**: Enterprise-grade authentication
- **VSCode Extension**: Generated extension files for IDE integration
- **Memory Engine**: Agent-aware context storage
- **MCP Server**: Natural language interface to AG2 tooling
- **Deployment Scripts**: Automated setup with NPM and UV support

### 🔧 Next Steps:
1. Install the Windows-Clippy-MCP server from the referenced repository
2. Run the deployment script to set up the complete integration
3. Configure Azure credentials for Key Vault integration
4. Install and test the VSCode extension
5. Start using AG2 agents with Windows-Clippy-MCP functionality

### 🌟 Key Benefits:
- **Seamless Integration**: Works with existing AG2 agent patterns
- **Enterprise Ready**: Azure AD and Key Vault support
- **Developer Friendly**: VSCode extension and natural language interface
- **Extensible**: MCP protocol allows for easy expansion
- **Secure**: Centralized credential management